In [ ]:
from quackosm.osm_extracts.bbbike import _load_bbbike_index
from quackosm.osm_extracts.geofabrik import _load_geofabrik_index
from quackosm.osm_extracts.osm_fr import _load_openstreetmap_fr_index
from quackosm.osm_extracts.extract import OsmExtractSource
import pandas as pd
from rich.tree import Tree
from rich.text import Text

# idx = _load_geofabrik_index()
# idx

In [ ]:
x = _load_geofabrik_index()
x

In [ ]:
x.loc[x["id"].str.startswith("us/"), "parent"] = "us"

In [ ]:
def create_branch(parent_id: str, tree: Tree, index: pd.DataFrame, depth: int = 0) -> None:
    matching_children = index[index.parent == parent_id].sort_values(by="name")
    # matched_ids = set(matching_children["id"])
    for matching_child in matching_children.to_dict(orient="records"):
        # text_filename = Text(path.name, "green")
        # text_filename.highlight_regex(r"\..*$", "bold red")
        # text_filename.stylize(f"link file://{path}")
        # file_size = path.stat().st_size
        # text_filename.append(f" ({decimal(file_size)})", "blue")
        # icon = "🐍 " if path.suffix == ".py" else "📄 "
        # tree.add(Text(icon) + text_filename)
        # branch = tree.add(matching_child["name"])
        branch = tree.add(f":globe_with_meridians: {matching_child['name']}")
        create_branch(matching_child["id"], branch, index, depth + 1)

    if depth == 0:
        loose_parents = set(index["parent"]).difference(index["id"]).difference([parent_id])
        for loose_parent in loose_parents:
            branch = tree.add(loose_parent)
            create_branch(loose_parent, branch, index, depth + 1)

    # if depth == 0:
    #     unmatched_children = index[~index.id.isin(matched_ids)].sort_values(by="name")

    # return matched_ids


root_id = OsmExtractSource.geofabrik.value
geofabrik_root = Tree(root_id)
create_branch(root_id, geofabrik_root, _load_geofabrik_index())

root_id = OsmExtractSource.bbbike.value
bbbike_root = Tree(root_id)
create_branch(root_id, bbbike_root, _load_bbbike_index())

root_id = OsmExtractSource.osm_fr.value
osm_fr_root = Tree(root_id)
create_branch(root_id, osm_fr_root, _load_openstreetmap_fr_index())

# idx = _load_bbbike_index()
# root_id = OsmExtractSource.bbbike.value

# idx = _load_openstreetmap_fr_index()
# root_id = OsmExtractSource.osm_fr.value


# def create_branch(parent_id: str, tree: Tree, index: pd.DataFrame) -> None:
#     matching_children = index[index.parent == parent_id].sort_values(by="name")
#     for matching_child in matching_children.to_dict(orient="records"):
#         branch = tree.add(matching_child["name"])
#         create_branch(matching_child["id"], branch, index)


# create_branch(root_id, root, idx)

In [ ]:
geofabrik_root

In [ ]:
bbbike_root

In [ ]:
osm_fr_root

In [ ]:
import pandas as pd


tree = {}

for _, row in idx.iterrows():
    if pd.isna(row.parent):
        continue
    elif row.parent not in tree:
        tree[row.parent] = []

    tree[row.parent].append(row.id)

tree

In [ ]:
"""
Demonstrates how to display a tree of files / directories with the Tree renderable.
"""

import os
import pathlib
import sys

from rich import print
from rich.filesize import decimal
from rich.markup import escape
from rich.text import Text
from rich.tree import Tree


def walk_directory(directory: pathlib.Path, tree: Tree) -> None:
    """Recursively build a Tree with directory contents."""
    # Sort dirs first then by filename
    paths = sorted(
        pathlib.Path(directory).iterdir(),
        key=lambda path: (path.is_file(), path.name.lower()),
    )
    for path in paths:
        # Remove hidden files
        if path.name.startswith("."):
            continue
        if path.is_dir():
            style = "dim" if path.name.startswith("__") else ""
            branch = tree.add(
                f"[bold magenta]:open_file_folder: [link file://{path}]{escape(path.name)}",
                style=style,
                guide_style=style,
            )
            walk_directory(path, branch)
        else:
            text_filename = Text(path.name, "green")
            text_filename.highlight_regex(r"\..*$", "bold red")
            text_filename.stylize(f"link file://{path}")
            file_size = path.stat().st_size
            text_filename.append(f" ({decimal(file_size)})", "blue")
            icon = "🐍 " if path.suffix == ".py" else "📄 "
            tree.add(Text(icon) + text_filename)


try:
    directory = pathlib.Path(".").resolve().parent.parent
except IndexError:
    print("[b]Usage:[/] python tree.py <DIRECTORY>")
else:
    tree = Tree(
        f":open_file_folder: [link file://{directory}]{directory}",
        guide_style="bold bright_blue",
    )
    walk_directory(pathlib.Path(directory), tree)
    print(tree)